In [179]:
import numpy as np 
import cv2 
import matplotlib.pyplot as plt 

In [180]:
Y = cv2.imread('Y.JPG',0)
cb = cv2.imread('Cb4.JPG',0)
cr = cv2.imread('Cr4.JPG',0)
Y=np.asarray(Y).astype(float)
cb=np.asarray(cb).astype(float)
cr=np.asarray(cr).astype(float)
cbu=cv2.resize(cb,(np.shape(Y)[1],np.shape(Y)[0])).astype(float)
cru=cv2.resize(cr,(np.shape(Y)[1],np.shape(Y)[0])).astype(float)
print(np.shape(Y))
print(np.shape(cb))
print(np.shape(cr))
print(np.shape(cbu))
print(np.shape(cru))

(622, 960)
(156, 240)
(156, 240)
(622, 960)
(622, 960)


In [181]:
def gaussianKernel(size_x,size_y, sigma_x,sigma_y,sampling_rate):
    if size_x%2==0 or size_y%2==0:
        print('Error: Kernel dimensions can only be odd')
        return
    center_x,center_y=size_x//2,size_y//2
    kernel =  np.fromfunction(lambda i, j:  np.exp(-((((i-center_x)/(sigma_x*sampling_rate))**2)/2+(((j-center_y)/(sigma_y*sampling_rate))**2)/2)), (size_x, size_y))
    return kernel 

In [187]:
kernel=gaussianKernel(5,5,5,5,4)

[[0.03980033 0.03994986 0.03999983 0.03994986 0.03980033]
 [0.03994986 0.04009995 0.04015011 0.04009995 0.03994986]
 [0.03999983 0.04015011 0.04020032 0.04015011 0.03999983]
 [0.03994986 0.04009995 0.04015011 0.04009995 0.03994986]
 [0.03980033 0.03994986 0.03999983 0.03994986 0.03980033]]


In [183]:
out_b=np.zeros((np.shape(Y)))
out_r=np.zeros((np.shape(Y)))
kernel_x,kernel_y=np.shape(kernel)
Y_pad=np.pad(Y,((kernel_x//2,kernel_x//2),(kernel_y//2,kernel_y//2)))
cbu_pad=np.pad(cbu,((kernel_x//2,kernel_x//2),(kernel_y//2,kernel_y//2)))
cru_pad=np.pad(cru,((kernel_x//2,kernel_x//2),(kernel_y//2,kernel_y//2)))
dim_x,dim_y=np.shape(Y)
range_sigma=5
for i in range(dim_x):
    for j in range(dim_y):
        center_x,center_y=(2*i+kernel_x)//2,(2*j+kernel_y)//2        
        range_kernel=np.exp(-0.5*((Y_pad[i:i+kernel_x,j:j+kernel_y]-Y_pad[center_x,center_y])/range_sigma)**2)
        final_kernel=range_kernel*kernel
        final_kernel=final_kernel/np.sum(final_kernel)
        out_b[i,j]=np.sum(cbu_pad[i:i+kernel_x,j:j+kernel_y]*final_kernel)
        out_r[i,j]=np.sum(cru_pad[i:i+kernel_x,j:j+kernel_y]*final_kernel)

In [184]:
final_img_YCbCr=np.array([Y,out_r,out_b]).astype(np.uint8).transpose(1,2,0)
final_img_bgr = cv2.cvtColor(final_img_YCbCr, cv2.COLOR_YCrCb2BGR)
cv2.imwrite('daddu.jpg',final_img_bgr)

True